# Выбор локации для скважины

## Описание проекта

Вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.


## Цель исследования

Построить модель для определения региона, где добыча принесёт наибольшую прибыль

## Инструкция по выполнению проекта

Загрузите и подготовьте данные. Поясните порядок действий.

Обучите и проверьте модель для каждого региона:
- Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
- Обучите модель и сделайте предсказания на валидационной выборке.
- Сохраните предсказания и правильные ответы на валидационной выборке.
- Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
- Проанализируйте результаты.
Подготовьтесь к расчёту прибыли:
- Все ключевые значения для расчётов сохраните в отдельных переменных.
- Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
- Напишите выводы по этапу подготовки расчёта прибыли.
Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- Выберите скважины с максимальными значениями предсказаний.
- Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
- Рассчитайте прибыль для полученного объёма сырья.
Посчитайте риски и прибыль для каждого региона:
- Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

## Описание данных

Данные геологоразведки трёх регионов находятся в файлах:
- geo_data_0.csv.
- geo_data_1.csv.
- geo_data_2.csv.
  
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

### Загрузка и подготовка данных

In [ ]:
# Загрузим необходимые библиотеки и функции

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [ ]:
# отключим некритические предупреждения в коде
warnings.filterwarnings("ignore")

In [ ]:
# сбросим ограничение на количество выводимых столбцов, что бы просмотреть все столбцы
pd.set_option('display.max_columns', None)

In [ ]:
# Загрузим данные

geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')


# Дабы в дальнейшем не повторять одни и те же действия на каждом датасете, объеденим их в список.
# А действия, которые надо выполнять, авернем в функции

datas= [geo_data_0, geo_data_1, geo_data_2]

In [ ]:
print(datas)

[          id        f0        f1        f2     product
0      txEyH  0.705745 -0.497823  1.221170  105.280062
1      2acmU  1.334711 -0.340164  4.365080   73.037750
2      409Wp  1.022732  0.151990  1.419926   85.265647
3      iJLyR -0.032172  0.139033  2.978566  168.620776
4      Xdl7t  1.988431  0.155413  4.751769  154.036647
...      ...       ...       ...       ...         ...
99995  DLsed  0.971957  0.370953  6.075346  110.744026
99996  QKivN  1.392429 -0.382606  1.273912  122.346843
99997  3rnvd  1.029585  0.018787 -1.348308   64.375443
99998  7kl59  0.998163 -0.528582  1.583869   74.040764
99999  1CWhH  1.764754 -0.266417  5.722849  149.633246

[100000 rows x 5 columns],           id         f0         f1        f2     product
0      kBEdx -15.001348  -8.276000 -0.005876    3.179103
1      62mP7  14.272088  -3.475083  0.999183   26.953261
2      vyE1P   6.263187  -5.948386  5.001160  134.766305
3      KcrkZ -13.081196 -11.506057  4.999415  137.945408
4      AHL4O  12.702195  -

In [ ]:
# Выводит информацию по датасету

def view(data):

    print('Обзор данных', '\n')
    print(data.head(10))
    print('\n')
    print('Общая информация ', '\n')
    print(data.info())
    print('\n')
    print('Описательная статистика', '\n')
    print(data.describe())
    print('\n')
    print('Дубикаты', '\n')
    print(data.duplicated().sum())
    print('-' * 65)
    print('\n' * 3)

In [ ]:
#Так как data - это список с датасетами, пройдемся по нему, вывода информацию по каждому датасету

for data in datas:
    view(data)

Обзор данных 

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
5  wX4Hy  0.969570  0.489775 -0.735383   64.741541
6  tL6pL  0.645075  0.530656  1.780266   49.055285
7  BYPU6 -0.400648  0.808337 -5.624670   72.943292
8  j9Oui  0.643105 -0.551583  2.372141  113.356160
9  OLuZU  2.173381  0.563698  9.441852  127.910945


Общая информация  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), 

Пропусков нет. Дубликатов нет. Аномалий и выбросов не наблюдается. Данные в отличном состоянии!
Нужно лишь удалить столбец ID, так как он не несет никакой ценности для дальнейшх исследований

In [ ]:
for i in range(len(datas)):
    datas[i] = datas[i].drop(columns=['id'], axis=0)

### Итог

- Прочитали данные и объеденили их.

- Проверили корректность данных.

- Удалили ненужный столбец.

Данные, как и сказанно выше, в отличном состоянии и не требуют большей предобработки.

## Обучение и проверка модели

### Обучение и проверка модели

In [ ]:
# разбивает на тренировочные и тестовые выборки и отделяет целевой признак

def data_split(data):

    train, test = train_test_split(data, test_size=0.25, random_state=12345)

    features_train = train.drop(columns=['product'], axis=0)
    target_train = train['product']

    features_test = test.drop(columns=['product'], axis=0)
    target_test= test['product']

    return features_train, target_train, features_test, target_test

In [ ]:
# создадим выборки по каждому датасету

features_train_0, target_train_0, features_test_0, target_test_0 = data_split(datas[0])
features_train_1, target_train_1, features_test_1, target_test_1 = data_split(datas[1])
features_train_2, target_train_2, features_test_2, target_test_2 = data_split(datas[2])

In [ ]:
# Стандартизируем данные

scaler = StandardScaler()

scaler.fit(features_train_0)
features_train_0 = scaler.transform(features_train_0)
features_test_0 = scaler.transform(features_test_0)

scaler.fit(features_train_1)
features_train_1 = scaler.transform(features_train_1)
features_test_1 = scaler.transform(features_test_1)

scaler.fit(features_train_2)
features_train_2 = scaler.transform(features_train_2)
features_test_2 = scaler.transform(features_test_2)

In [ ]:
model = LinearRegression()

# Обучает и предсказывает по каждому датасету

def learning_and_predicted(features_train, target_train, features_test, target_test):

    model.fit(features_train, target_train)
    predict = pd.Series(model.predict(features_test), target_test.index)

    return predict

In [ ]:
predict_0 = learning_and_predicted(features_train_0, target_train_0, features_test_0, target_test_0)
predict_1 = learning_and_predicted(features_train_1, target_train_1, features_test_1, target_test_1)
predict_2 = learning_and_predicted(features_train_2, target_train_2, features_test_2, target_test_2)

In [ ]:
predictions = [predict_0, predict_1, predict_2]
targets_test = [target_test_0, target_test_1, target_test_2]

for i in range(len(predictions)):

    print('Регион', i+1) # Дабы "Регион 0" не резал глаз

    mse = mean_squared_error(predictions[i], targets_test[i])
    rmse = mse**0.5

    print('Предсказанный средний запас: {:,.2f}'.format(predictions[i].mean()))
    print('RMSE: {:,.2f}'.format(rmse), '\n')

Регион 1
Предсказанный средний запас: 92.59
RMSE: 37.58 

Регион 2
Предсказанный средний запас: 68.73
RMSE: 0.89 

Регион 3
Предсказанный средний запас: 94.97
RMSE: 40.03 



### Итог

В первом и третьем регионах модель ошибается довольно таки сильно, при этом во втором регионе ошибки в предсказаниях крайне малы.
Для выдвижения гипотезы о том, почему так, нет достаточных данных. Потому просто приму как есть

## Подготовка к расчёту прибыли

### Создадим константы

In [ ]:
# кол-во используемых точек

TOTAL_OBJECTS_= 200

# Бюджет на регион

TOTAL_BUDJET_= 10e9

# Цена за тысячу баррелей

THOUSEND_BARREL_PRICE_= 450e3

# Значение допустимого риска
RISK = 0.025


### Рассчитаем достаточный объем для безубыточности

In [ ]:
# Цена одной скважины

object_price_ = TOTAL_BUDJET_/TOTAL_OBJECTS_

# Объем для окупаемости

min_volume_ = object_price_/THOUSEND_BARREL_PRICE_


print('Цена одной скважины: {:.2f}'.format(object_price_))
print('Минимальный объем в скважине, для окупаемости: {:.2f} bar'.format(min_volume_*1000)) #

Цена одной скважины: 50000000.00
Минимальный объем в скважине, для окупаемости: 111111.11 bar


### Создадим функцию для расчета прибыли



In [ ]:
def revenue(target, probabilities, count): # Считаем прибыль
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return selected.sum() * thousand_barrel_price_ - total_budjet_

state = np.random.RandomState(12345)

### Итог

Как видим, средние объемы в регионах ниже, чем минимальный объем для окупаемости, особенно во втором регионе - почти в два раза.


## Расчёт прибыли и рисков

In [ ]:
# Расчитаем выручку по каждому региону

for i in range(len(targets_test)):
    values = []
    counter = 0
    for _ in range(1000):
        target_subsample = targets_test[i].sample(n = 500, replace=True, random_state=state)
        probs_subsample = predictions[i][target_subsample.index]
        reven = revenue(target_subsample, probs_subsample, total_objects_)
        values.append(reven)
        if reven < 0:
            counter += 1
    name = 'регион №' + str(i+1)
    values = pd.Series(values)
    lower = values.quantile(RISK)
    upper = values.quantile(0.975)
    mean =  values[(lower < values) & (values < upper)].mean()

    print(name.upper(),'\n')
    print("Средняя выручка:{:.2f}".format(mean))
    print ('95% доверительный интервал: {:.2f} - {:.2f}'.format(lower, upper))
    print('Риск убытков: {:.2%}'.format(counter / 1000))
    print('\n')


РЕГИОН №1 

Средняя выручка:426280874.05
95% доверительный интервал: -102090094.84 - 947976353.36
Риск убытков: 6.00%


РЕГИОН №2 

Средняя выручка:517022253.90
95% доверительный интервал: 128123231.43 - 953612982.07
Риск убытков: 0.30%


РЕГИОН №3 

Средняя выручка:420046101.84
95% доверительный интервал: -115852609.16 - 989629939.84
Риск убытков: 6.20%




## Общий итог

По результатам исследования могу рекомендовать 2 регион:

- риск убытков 1%
- средняя предполагаемая выручка - 499279008.41 - так же выше, чем в остальных регионах
- RMSE по предсказаниям в этом регионе самая низкая - всего 0.89.

Так как ошибки в предсказаниях по другим регионам намного выше, можем предположить, что и предсказанные уровни прибыли по регионам могут сильно отличаться от реальных.
Да и риск убытков может быть выше